# Build your own Quantum Architecture with tket

Make sure you follow first the getting started guide for `pytket` available in the Hackathon repo.
We recommend you first go through the example notebooks listed in the getting started guide.

To run the following, you will need to have both `pytket` and `pytket-qiskit` installed, as well as standard
python libraries such as `numpy`. If you can successfully run the following cell, you are good to go:


In [1]:
import numpy as np
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox, OpType, Op
from pytket.passes import (
    RebaseQuil, RebaseIBM, CliffordSimp, SynthesiseIBM, SequencePass, DecomposeBoxes, FullPeepholeOptimise, EulerAngleReduction
)
from pytket.extensions.qiskit import tk_to_qiskit

## Universal gatesets
### Arbitrary one-qubit gates
Quantum circuits are made of quantum gates. Quantum gates are unitary matrices, i.e. linear operations with determinant of norm one.
In principle, any unitary matrix acting on a set of qubits
could define such a gate (and thus in theory, any circuit can itself be seen
as a single quantum gate, given that circuit are always defined as a composition of simpler quantum gates).

In `pytket`, you can create a gate on a single qubit from an arbitrary unitary as follows:

In [2]:
def random_2dim_unitary():
    """ returns a random 2x2 unitary

    The details of this function do not matter, they are just meant to produce some unitary matrix
    Note: this implementation is for demonstration only -- this does not generate matrices uniformly
    """
    # a random 2dim vector
    v = np.random.rand(2) + 1j * np.random.rand(2)
    # normalise it
    v /= np.linalg.norm(v)
    # choose an orthogonal vector
    w = np.array([-v[1].conj(), v[0].conj()])
    # return matrix [v, w]
    return np.stack([v, w])

U = random_2dim_unitary()
print(f"Got random unitary:\n{U}")
print(f"det = {np.linalg.det(U)}")

# create single qubit gate given by U
Ugate = Unitary1qBox(U)

# Define empty circuit with just one qubit
circ = Circuit(1)
# add Ugate on qubit 0
circ.add_unitary1qbox(Ugate, 0)

Got random unitary:
[[ 0.58743299+0.36845745j  0.2194812 +0.68628682j]
 [-0.2194812 +0.68628682j  0.58743299-0.36845745j]]
det = (0.9999999999999999-5.551115123125783e-17j)


[Unitary1qBox q[0]; ]

However, when it comes to running circuits on actual machines, it makes sense to restrict the set of gates under consideration.
Indeed, there are a lot of rather limited set of gates that we know can be used to produce arbitrary computations through composition of these gates.
We call this property the _universality_ of the gateset.

We can for example express the above random gate as a composition of rotation gates:

In [3]:

# we define a tket pass that proceeds in two steps
#   1. decomposes the unitary gate Ugate with `DecomposeBoxes`
#   2. expresses the resulting circuit in terms of rotations with `RebaseQuil`
to_zx_rotations = SequencePass([
    DecomposeBoxes(),
    RebaseQuil()
])
# apply pass to the circuit
to_zx_rotations.apply(circ)

print(tk_to_qiskit(circ))

global phase: π
     ┌────────────┐┌────────────┐┌────────────┐
q_0: ┤ RZ(2.2719) ├┤ RX(1.6091) ├┤ RZ(2.8909) ├
     └────────────┘└────────────┘└────────────┘


We see that tket is able to decompose our original random gate as a composition of `Rx` and `Rz` gates, called `X`- and `Z`-rotations respectively.
We won't go into detail why these gates are called rotations -- for our purposes it is enough to consider this a weird naming convention.
If you are interested in learning more, looking up "Bloch sphere" should be a good place to start.

### Generalising to several qubits
For good performance, it is critical to look for gatesets that can be implemented easily on hardware, while at the same time giving
us enough expressivity to perform computations efficiently.
One of the most widely used sets of gates is a simple extension of the rotations above.
Indeed, such rotations are enough to express any computation on a single qubit --
and this can even be extended to any computation on arbitrary many qubits, using a single
additional gate!
In the following this additional gate will be the controlled Z-gate -- much of the literature uses
either this, or equivalently, the controlled X-gate, also known as the controlled NOT (CNot) gate.

Let us repeat the code above, generating a random unitary and then decomposing it into a circuit.
This time, however, we are interested in a two-qubit circuit: this means our unitaries are now 4x4 dimensional.

In [4]:
def random_4dim_unitary():
    """ returns a random 4x4 unitary

    The details of this function do not matter, they are just meant to produce some unitary matrix
    Note: this implementation is for demonstration only -- this does not generate matrices uniformly
    """
    # a random 4dim vector
    v = np.random.rand(4) + 1j * np.random.rand(4)
    # normalise it
    v /= np.linalg.norm(v)
    # choose three orthogonal vectors w1, w2, w3
    ws = [v]
    while len(ws) < 4:
        new_w = np.random.rand(4) + 1j * np.random.rand(4)
        # make new_w orthogonal to all previous vectors using GramSchmidt
        for w in ws:
            new_w -= np.vdot(w, new_w)*w
        new_w /= np.linalg.norm(new_w)
        ws.append(new_w)
    # return matrix [v, w1, w2, w3]
    return np.stack(ws)

U = random_4dim_unitary()

print(f"Got random unitary:\n{U}")
print(f"det = {np.linalg.det(U)}")

# create single qubit gate given by U
Ugate = Unitary2qBox(U)

# Define empty circuit with two qubits
circ = Circuit(2)
# add Ugate on qubits 0 and 1
circ.add_unitary2qbox(Ugate, 0, 1)

# we now use the same pass as before:
to_zx_rotations.apply(circ)
print(tk_to_qiskit(circ))

Got random unitary:
[[ 0.56281451+0.15606603j  0.08863735+0.70386593j  0.01779412+0.35174953j
   0.10372717+0.14420716j]
 [-0.24059424+0.4302829j   0.19151001-0.21191995j  0.54233288+0.34392159j
  -0.03787221+0.51141331j]
 [ 0.39649492+0.41639649j  0.48454857-0.3728714j  -0.45577172-0.10464556j
  -0.2752844 +0.03354535j]
 [-0.13187053+0.26051902j  0.09571109-0.17930148j -0.13576837+0.47585399j
   0.59558599-0.52329497j]]
det = (-0.8013089989607016+0.5982506900828434j)
global phase: 6.1229
     ┌────────────┐┌────────────┐┌────────────┐                      »
q_0: ┤ RZ(4.8676) ├┤ RX(1.2596) ├┤ RZ(11.199) ├──────────────────────»
     ├────────────┤├────────────┤├────────────┤┌─────────┐┌─────────┐»
q_1: ┤ RZ(1.8243) ├┤ RX(2.9666) ├┤ RZ(1.8751) ├┤ RZ(π/2) ├┤ RX(π/2) ├»
     └────────────┘└────────────┘└────────────┘└─────────┘└─────────┘»
«                   ┌──────────┐┌─────────┐┌────────────┐               »
«q_0: ────────────■─┤ RZ(7π/2) ├┤ RX(π/2) ├┤ RZ(1.3657) ├───────────────»
«  

As you can see, tket has managed to express this arbitrary unitary using only rotations and the `CZ`
gate, which is represented as two linked black boxes in the above circuit representation.
We are thus able to express arbitrary two-qubit operations in this gateset!
In fact, this gateset is indeed universal, that is, any operation on arbitrary many qubits can be expressed by these gates.
While in theory we know how to decompose any such unitary transformation on $n$ qubits into circuit form, doing so
efficiently actually turns out to be a really hard problem and is the subject of active research.

### Simplifying our circuit
The circuit we have just obtained for our two-qubit unitary is horribly long and ugly.
This is where tket's circuit simplification capabilities come in.
We can extend the `to_zx_rotations` pass that we defined earlier to include some optimisation:

In [5]:
# for our purposes, this is sufficient:
# we simply add a EulerAngleReduction pass at the end which finds the optimal
# angles for the rotations to avoid duplicates
#
# use FullPeepholeOptimise as a general purpose optimiser that should work well on
# pretty much any circuit
to_zx_optimised = SequencePass([
    DecomposeBoxes(),
    # FullPeepholeOptimise(),  <-- add this for general-purpose optimisation
    RebaseQuil(),
    EulerAngleReduction(OpType.Rx, OpType.Rz)
])

# Define empty circuit with two qubits
circ = Circuit(2)
# add Ugate on qubits 0 and 1
circ.add_unitary2qbox(Ugate, 0, 1)

# we now use the same pass as before:
to_zx_rotations.apply(circ)
print(tk_to_qiskit(circ))

global phase: 6.1229
     ┌────────────┐┌────────────┐┌────────────┐                      »
q_0: ┤ RZ(4.8676) ├┤ RX(1.2596) ├┤ RZ(11.199) ├──────────────────────»
     ├────────────┤├────────────┤├────────────┤┌─────────┐┌─────────┐»
q_1: ┤ RZ(1.8243) ├┤ RX(2.9666) ├┤ RZ(1.8751) ├┤ RZ(π/2) ├┤ RX(π/2) ├»
     └────────────┘└────────────┘└────────────┘└─────────┘└─────────┘»
«                   ┌──────────┐┌─────────┐┌────────────┐               »
«q_0: ────────────■─┤ RZ(7π/2) ├┤ RX(π/2) ├┤ RZ(1.3657) ├───────────────»
«     ┌─────────┐ │ ├─────────┬┘├─────────┤└┬─────────┬─┘┌─────────────┐»
«q_1: ┤ RZ(π/2) ├─■─┤ RZ(π/2) ├─┤ RX(π/2) ├─┤ RZ(π/2) ├──┤ RZ(0.75979) ├»
«     └─────────┘   └─────────┘ └─────────┘ └─────────┘  └─────────────┘»
«                                         ┌─────────┐┌─────────┐┌─────────┐»
«q_0: ──────────────────────────────────■─┤ RZ(π/2) ├┤ RX(π/2) ├┤ RZ(π/2) ├»
«     ┌─────────┐┌─────────┐┌─────────┐ │ ├─────────┤├─────────┤├─────────┤»
«q_1: ┤ RZ(π/2) ├┤ RX(π

## Your task

Enough of me talking. Now is the time for you to start playing around, and do your bit for quantum research :)

You have the job of the architecture designer for our newest quantum computer.
Your task is to try and come up with a gatesets that results in circuits as short as possible: the shorter the circuit, the more likely we can run it sucessfully on our quantum device, and the more money
you will make.

### Out-of-the-box support
tket comes with a host of supported gatesets, inspired by architectures of real machines.
You can find all the suported gatesets by looking at the compiler passes with names that start with
"Rebase" on [this page](https://cqcl.github.io/pytket/build/html/passes.html).
Compiler passes will take your circuit and rewrite it such that it only contains
gates in the respective gateset: you can use them as follows
```python
pass = RebaseSomething()
pass.apply(circuit)
```
Examples of such passes are `RebaseQuil` (which we used earlier), `RebaseCirq` and many others.
The names come from the architecture or platform these gatesets are used in.

Along with rebase passes, it makes sense to use some simplification passes that will make sure your output circuit
is as short as it can be.
For your convenience, here is a sequence that should work for pretty much anything:

In [6]:
# choose your target gateset
RebaseXYZ = RebaseIBM

compile_pass = SequencePass([
    DecomposeBoxes(),
    FullPeepholeOptimise(),
    CliffordSimp(False),
    SynthesiseIBM(),
    RebaseXYZ(),
    # if you use rotation gates, this can get rid of some additional gates
    # EulerAngleReduction(OpType.Rx, OpType.Rz) 
])

#### Where is this sequence pass coming from, and how do I come up with this?

For all supported devices, tket will suggest a recommended compilation pass, which you can always obtain using
`device.default_compilation_pass()`.
For example, for an IBM device you obtain the following.
You can see the result is similar to the pass defined above.

You can get more information on compiler passes and what they do [here](https://cqcl.github.io/pytket/build/html/manual_compiler.html).

In [7]:
from pytket.extensions.qiskit import IBMQBackend

# IBM has multiple devices. We need to specify which one we want
# this will yield an error if you do not have a API key registered with qiskit
# (you can get this for free by following the instructions here:
# https://qiskit.org/documentation/install.html#access-ibm-quantum-systems)
device_name = "ibmq_16_melbourne"
device = IBMQBackend(device_name)

compile_pass = device.default_compilation_pass(optimisation_level=2)
for p in compile_pass.get_sequence():
    print(p.get_config()['name'])

DecomposeBoxes
FullPeepholeOptimise
SequencePass
CliffordSimp
SynthesiseIBM
RebasePass
RemoveRedundancies


### Circuits
To try out these different passes, I have added a series of circuits to the repo.
They are available in the `circuits` folder.
They are all saved in the QASM format, which is a standard circuit format supported by virtually all
quantum software.

You can load such a circuit as follows

In [8]:
from pytket.qasm import circuit_from_qasm

circ = circuit_from_qasm('circuits/small_1.qasm')
print(tk_to_qiskit(circ))

     ┌────────────┐┌─────────────┐┌────────────┐┌─────────┐┌─────────┐»
q_0: ┤ RZ(4.7984) ├┤ RX(0.84427) ├┤ RZ(5.0173) ├┤ RZ(π/2) ├┤ RZ(π/2) ├»
     ├────────────┤└┬────────────┤├────────────┤└─────────┘└─────────┘»
q_1: ┤ RZ(3.7423) ├─┤ RX(2.4257) ├┤ RZ(5.2854) ├──────────────────────»
     └────────────┘ └────────────┘└────────────┘                      »
c: 2/═════════════════════════════════════════════════════════════════»
                                                                      »
«     ┌─────────┐┌─────────┐   ┌─────────┐  ┌─────────┐  ┌─────────┐»
«q_0: ┤ RX(π/2) ├┤ RZ(π/2) ├─■─┤ RZ(π/2) ├──┤ RX(π/2) ├──┤ RZ(π/2) ├»
«     └─────────┘└─────────┘ │ ├─────────┴┐┌┴─────────┴─┐├─────────┤»
«q_1: ───────────────────────■─┤ RZ(3π/2) ├┤ RX(5.8818) ├┤ RZ(π/2) ├»
«                              └──────────┘└────────────┘└─────────┘»
«c: 2/══════════════════════════════════════════════════════════════»
«                                                                   »
«     

Have a look at all the circuits available and see what their compiled version looks like.

Feel free to also construct your own circuits.
Defining a simple circuit on 4 qubits, for example, is as simple as `Circuit(4).CX(1, 2).Rx(0.4, 3)`,
where the `CX` and `Rx` calls define gates in your circuit.
A good overview of how to construct circuits is given [here](https://cqcl.github.io/pytket/build/html/manual_circuit.html). 

## What next?
Once you get a feeling about differences between gatesets and what tket passes do, there 
are several directions you can explore

 1. You can define your own custom gatesets, using the `RebaseCustom` pass from tket. Look at the [documentation](https://cqcl.github.io/pytket/build/html/passes.html#pytket.passes.RebaseCustom) to understand what it does, and see if you can come up with interesting gatesets. You will find the list of all gates that you can use [here](https://cqcl.github.io/pytket/build/html/optype.html)
 2. Beside limited gatesets, quantum computers also have further architecture constraints that limit the circuits that can be run. One such constraint is that two-qubit gates (such as CNots or CZ gates) can only be performed between certain gates. tket deals with that using the [CXMappingPass](https://cqcl.github.io/pytket/build/html/passes.html#pytket.passes.CXMappingPass), which takes arbitrary circuits and transforms them to make sure that there are only CNots between allowed qubits.
 Explore how that affects your circuits!
 3. Beyond two-qubit gates, one can also imagine using gates that act on 3 qubits or more. How would you deal with these? Can you use `RebaseCustom` for that purpose? What are the difficulties?

## Relevant references
- tket getting started: https://cqcl.github.io/pytket/build/html/getting_started.html
- tket demos: https://github.com/CQCL/pytket/tree/master/examples
- Circuit construction: https://cqcl.github.io/pytket/build/html/manual_circuit.html
- Compilation: https://cqcl.github.io/pytket/build/html/manual_compiler.html

Don't hesitate to reach out to me if you have any questions or need a bit of help! Good luck!

Luca

[luca.mondada@cambridgequantum.com](mailto:luca.mondada@cambridgequantum.com) (or on slack)